In [1]:
!pip install transformers -q
!pip install streamlit -q
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import openai, requests, os

# Intro Text
st.markdown('# PDF summarization using Generative AI')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

def summarise_file(prompt,model = 'gpt-3.5-turbo'):
  # Connect to API endpoint
  openai.api_key = ''
  URL = 'https://api.openai.com/v1/chat/completions'

  parameters = {
      'model':model,
      'messages': [{'role':'user', 'content':prompt}],
      'temperature': 1,
      'top_p': 1
  }

  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {openai.api_key}'
  }

  response = requests.post(URL, headers=headers, json=parameters, stream=False)
  json_response = response.json()
  summary = json_response['choices'][0]['message']['content'].strip()
  return summary


def keywords(prompt,model = 'gpt-3.5-turbo'):
  # Connect to API endpoint
  openai.api_key = ''
  URL = 'https://api.openai.com/v1/chat/completions'

  parameters = {
      'model':model,
      'messages': [{'role':'user', 'content':prompt}],
      'temperature': 1,
      'top_p': 1
  }

  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {openai.api_key}'
  }

  response = requests.post(URL, headers=headers, json=parameters, stream=False)
  json_response = response.json()
  summary = json_response['choices'][0]['message']['content'].strip()
  return summary

if input_file:
  prompt = f'''
  ### TASK
  You are an expert in text analysis and summarisation with research papers. Your task is to analyse a research paper I'm going to give you
  and provide a detailed summary that is most representative of its content. You must use the Key Concept Clarity method to read the paper and
  highlight all of the key talking points of the paper.
  ### INPUTS
  Research Paper: {input_file}
  ### OUTPUT
  The output should be structured so as to identify and concisely explain the main ideas in the document. Each section should consist of the
  name of the talking point, a detailed definition of the talking point and a short summary of the talking point.
  Aim to provide at least five talking points. Here is how each talking point should be structured:
  Title: Name of the talking point
  Explanation: Define in detail what is the talking point
  Overview: Provide a short summary of the talking point including its challenges ,if any, and findings.
  '''
  text_area = summarise_file(prompt)
  st.success('Summary lodaing, please wait...')

  # Show Summary
  st.success('Summary ready!')
  st.text_area(label ="",value= text_area, placeholder="Please upload a PDF to get it's summary", height = 400)
  st.markdown('')

  prompt = f'''
  ### TASK
  You are an expert in creating abstracts from research papers and finding similar abstracts. Your task is to analyse a research paper I'm going
  to give you and create an abstract that is most representative of its content. Then you're going to searcg https://arxiv.org/ and return the
  names and urls of the top 5 results.
  ### INPUTS
  Summary: {input_file}
  ### OUTPUT
  For each result, provide a short explanation of why you think the two abstracts are similar.
  '''
  test = keywords(prompt)
  st.success('Recommendations lodaing, please wait...')
  # Show Summary
  st.success('Recommendations ready!')
  st.text_area(label ="",value= test, placeholder="Please upload a PDF to get it's summary", height = 400)
  st.markdown('')

else:
  st.text_area(label ="",value= "Please upload a PDF to get it's summary", placeholder="Please upload a PDF to get it's summary", height = 400)

Writing app.py


In [4]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 5.096s


In [5]:
!curl ipv4.icanhazip.com

34.106.218.62


In [8]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

npx: installed 22 in 1.909s
your url is: https://shaky-showers-stay.loca.lt
/root/.npm/_npx/1058/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:41117 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/1058/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
